In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, DatasetDict

# 모델 및 토크나이저 설정
model_name = "Qwen/Qwen2-7B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 정확도를 위한 bfloat16 사용
)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    quantization_config=quantization_config,
)

# LoRA 설정
tokenizer = AutoTokenizer.from_pretrained(model_name)

lora_configs = [
    LoraConfig(
        r=64,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "o_proj", "lm_head", "up_proj", "down_proj", "gate_proj"],
        lora_dropout=0.1,
        bias="none"
    ),
    LoraConfig(
        r=64,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "o_proj", "lm_head", "up_proj", "down_proj", "gate_proj"],
        lora_dropout=0.1,
        bias="none"
    )
]



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
from datasets import load_from_disk
combined_dataset = load_from_disk("data/multi_lora1_economic_data_v1")
combined_dataset_2 = load_from_disk("data/multi_lora2_finance_math_data_v1")
# 데이터셋 정의: 각 LoRA에 서로 다른 데이터셋 사용
datasets = [
    combined_dataset.select(range(0, 100)), 
    combined_dataset_2.select(range(0, 100)) # LoRA 1 데이터셋
    # load_dataset("imdb", split="train[:50%]"),  # LoRA 2 데이터셋
]


import pandas as pd
from datasets import Dataset,concatenate_datasets
from datasets import load_dataset
prompt_format = """아래는 작업을 설명하는 지침과 추가적인 맥락을 제공하는 입력이 함께 제공된 내용입니다. 요청을 적절히 완료하는 응답을 작성하라.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass





# 데이터셋 전처리
# def preprocess_dataset(dataset, instruction_key, response_key):
#     # 프롬프트 포맷팅
#     dataset = dataset.map(
#         lambda examples: formatting_prompts_func(examples, instruction_key, response_key),
#         batched=True
#     )
#     # 토크나이징
#     return dataset.map(
#         lambda examples: tokenizer(
#             examples["formatted_text"],
#             padding="max_length",
#             truncation=True,
#             max_length=1000
#         ),
#         batched=True
#     )
tokenized_dataset_list = []
for i in datasets:

    combined_dataset = i.map(formatting_prompts_func, batched = True)
    # 데이터셋 전처리: 각 LoRA에 맞는 데이터셋 정의
    def tokenize_function(examples):
        return tokenizer(examples["formatted_text"], padding="max_length", truncation=True, max_length=300)

    tokenized_dataset = combined_dataset.map(tokenize_function, batched=True)
    tokenized_dataset_list.append(tokenized_dataset)

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 데이터 콜레이터 설정
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
# 개별 LoRA 모델 학습 함수
def train_single_lora(lora_model, tokenized_dataset):
    

    training_args = TrainingArguments(
    output_dir="./qwen2-7b-qlora_4",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=3,
    learning_rate=10e-5,
    num_train_epochs=5,
    logging_steps=10,
    fp16=True,
    push_to_hub=False,
    save_steps= -1,

    )

    # Trainer 생성
    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    # 학습 시작
    trainer.train()


# 각 LoRA 모델 학습
lora_models = []
for i, config in enumerate(lora_configs):
    print(f"Training LoRA {i+1} on Dataset {i+1}")
    lora_model = get_peft_model(model, config)
    train_single_lora(lora_model, tokenized_dataset_list[i])  # 각 LoRA에 다른 데이터셋 전달
    lora_models.append(lora_model)



Training LoRA 1 on Dataset 1


  0%|          | 0/80 [00:00<?, ?it/s]

c:\Users\son\anaconda3\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:623: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.3552, 'grad_norm': 0.4804959297180176, 'learning_rate': 8.75e-05, 'epoch': 0.6}
{'loss': 1.019, 'grad_norm': 0.46917712688446045, 'learning_rate': 7.500000000000001e-05, 'epoch': 1.2}
{'loss': 0.9086, 'grad_norm': 0.5039673447608948, 'learning_rate': 6.25e-05, 'epoch': 1.8}
{'loss': 0.8166, 'grad_norm': 0.5528528690338135, 'learning_rate': 5e-05, 'epoch': 2.4}
{'loss': 0.7357, 'grad_norm': 0.6268351674079895, 'learning_rate': 3.7500000000000003e-05, 'epoch': 3.0}
{'loss': 0.6506, 'grad_norm': 0.7376654744148254, 'learning_rate': 2.5e-05, 'epoch': 3.6}
{'loss': 0.6126, 'grad_norm': 0.6690766215324402, 'learning_rate': 1.25e-05, 'epoch': 4.2}
{'loss': 0.5554, 'grad_norm': 0.7080705165863037, 'learning_rate': 0.0, 'epoch': 4.8}


c:\Users\son\anaconda3\Lib\site-packages\peft\utils\save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'train_runtime': 79.7399, 'train_samples_per_second': 6.27, 'train_steps_per_second': 1.003, 'train_loss': 0.8317156314849854, 'epoch': 4.8}
Training LoRA 2 on Dataset 2


  0%|          | 0/80 [00:00<?, ?it/s]

{'loss': 1.7916, 'grad_norm': 0.5029221773147583, 'learning_rate': 8.75e-05, 'epoch': 0.6}
{'loss': 1.3961, 'grad_norm': 0.4826034903526306, 'learning_rate': 7.500000000000001e-05, 'epoch': 1.2}
{'loss': 1.3163, 'grad_norm': 0.5625383853912354, 'learning_rate': 6.25e-05, 'epoch': 1.8}
{'loss': 1.2299, 'grad_norm': 0.6647530198097229, 'learning_rate': 5e-05, 'epoch': 2.4}
{'loss': 1.1191, 'grad_norm': 0.7074246406555176, 'learning_rate': 3.7500000000000003e-05, 'epoch': 3.0}
{'loss': 1.0289, 'grad_norm': 0.7119176387786865, 'learning_rate': 2.5e-05, 'epoch': 3.6}
{'loss': 0.9827, 'grad_norm': 0.7793198823928833, 'learning_rate': 1.25e-05, 'epoch': 4.2}
{'loss': 0.9375, 'grad_norm': 0.8118708729743958, 'learning_rate': 0.0, 'epoch': 4.8}


c:\Users\son\anaconda3\Lib\site-packages\peft\utils\save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'train_runtime': 79.6321, 'train_samples_per_second': 6.279, 'train_steps_per_second': 1.005, 'train_loss': 1.2252683520317078, 'epoch': 4.8}


In [16]:
import torch
import torch.nn.functional as F
from torch import nn

# Simplified Dynamic LoRA Weighted Layer
class DynamicLoRAWeightLayer(nn.Module):
    def __init__(self, base_model, lora_models, device='cuda'):
        super().__init__()
        self.device = device
        self.base_model = base_model.to(device)
        self.lora_models = nn.ModuleList([model.to(device) for model in lora_models])
        self.num_loras = len(lora_models)
        
        # Freeze base model and LoRA models
        for param in self.base_model.parameters():
            param.requires_grad = False
        for lora_model in self.lora_models:
            for param in lora_model.parameters():
                param.requires_grad = False
        
        # Define a new layer between base model and LoRA models
        self.intermediate_layer = nn.Sequential(
            nn.Linear(self.base_model.config.hidden_size, self.base_model.config.hidden_size),
            nn.ReLU()
        ).to(device)

        # Define weight generator to determine the contribution of each LoRA model
        self.weight_generator = nn.Sequential(
            nn.Linear(self.base_model.config.hidden_size, 128),
            nn.ReLU(),
            nn.Linear(128, self.num_loras),
            nn.Softmax(dim=-1)
        ).to(device)

        print(f"Model architecture:")
        print(f"- Base model hidden size: {self.base_model.config.hidden_size}")
        print(f"- Number of LoRA models: {self.num_loras}")

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        input_ids = input_ids.to(self.device)
        attention_mask = attention_mask.to(self.device) if attention_mask is not None else None
        if labels is not None:
            labels = labels.to(self.device)
        
        # Get hidden states from base model
        with torch.no_grad():
            base_hidden_states = self.base_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True).hidden_states[-1]
        
        # Pass through intermediate layer
        intermediate_output = self.intermediate_layer(base_hidden_states)  # Shape: (batch_size, seq_len, hidden_size)
        avg_hidden_states = intermediate_output.mean(dim=1)  # Shape: (batch_size, hidden_size)
        
        # Compute dynamic weights for LoRA models
        dynamic_weights = self.weight_generator(avg_hidden_states)  # Shape: (batch_size, num_loras)
        
        # Get logits from each LoRA model
        all_logits = []
        for lora_model in self.lora_models:
            with torch.cuda.amp.autocast():
                logits = lora_model(input_ids=input_ids, attention_mask=attention_mask).logits
                all_logits.append(logits)
        
        # Stack logits and compute weighted sum
        stacked_logits = torch.stack(all_logits, dim=1)  # Shape: (batch_size, num_loras, seq_len, vocab_size)
        dynamic_weights = dynamic_weights.unsqueeze(-1).unsqueeze(-1)  # Shape: (batch_size, num_loras, 1, 1)
        weighted_logits = (stacked_logits * dynamic_weights).sum(dim=1)  # Shape: (batch_size, seq_len, vocab_size)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(weighted_logits.view(-1, weighted_logits.size(-1)), labels.view(-1))
        
        return {
            "loss": loss,
            "logits": weighted_logits,
            "weights": dynamic_weights.squeeze(-1).squeeze(-1)  # Shape: (batch_size, num_loras)
        }

# Weighted Layer 학습 함수
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from transformers import PreTrainedTokenizerBase

def preprocess_dataset(dataset, tokenizer: PreTrainedTokenizerBase):
    # 입력 텍스트를 토큰화하고 필요한 열만 유지하도록 전처리
    def tokenize_function(examples):
        return tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=100)
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return tokenized_dataset

def train_weighted_layer(weighted_layer, combined_dataset, tokenizer, eval_dataset=None):
    # 데이터셋 전처리
    combined_dataset = preprocess_dataset(combined_dataset, tokenizer)
    if eval_dataset is not None:
        eval_dataset = preprocess_dataset(eval_dataset, tokenizer)
    
    training_args = TrainingArguments(
        output_dir="./qwen2-7b-dynamic-weights",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=3,
        learning_rate=1e-4,
        num_train_epochs=2,
        logging_steps=10,
        fp16=True,
        push_to_hub=False,
        save_steps=-1,
        evaluation_strategy="steps" if eval_dataset else "no",
        eval_steps=100 if eval_dataset else None,
        load_best_model_at_end=True if eval_dataset else False,
        metric_for_best_model="eval_loss" if eval_dataset else None,
        warmup_steps=100,
        weight_decay=0.01,
        gradient_checkpointing=False,
        remove_unused_columns=False  # 오류 해결을 위해 수정된 부분
    )

    class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            outputs = model(**inputs)
            loss = outputs["loss"]
            
            if self.state.global_step % self.args.logging_steps == 0:
                weights = outputs["weights"]
                avg_weights = weights.mean(dim=0)
                weight_std = weights.std(dim=0)
                
                # 가중치 통계 로깅
                for i, (w, std) in enumerate(zip(avg_weights, weight_std)):
                    self.log({
                        f"model_{i}_weight_mean": w.item(),
                        f"model_{i}_weight_std": std.item()
                    })
            
            return (loss, outputs) if return_outputs else loss

    trainer = CustomTrainer(
        model=weighted_layer,
        args=training_args,
        train_dataset=combined_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    )

    trainer.train()
    trainer.save_model(output_dir="./qwen2-7b-dynamic-weights-final")

# 데이터셋 샘플링 및 병합
from datasets import concatenate_datasets

def sample_and_combine_datasets(datasets, num_samples=100):
    # 각 데이터셋에서 샘플링
    sampled_datasets = [
        dataset.select(torch.randint(0, len(dataset), (num_samples,)).tolist())
        for dataset in datasets
    ]
    
    # 샘플링된 데이터셋 병합
    combined_dataset = concatenate_datasets(sampled_datasets)
    
    return combined_dataset


# Dynamic LoRA Weighted Layer 학습 실행
print("Preparing Dynamic Weighted Layer...")
weighted_layer = DynamicLoRAWeightLayer(model, lora_models)

print("Sampling and Combining Datasets...")
combined_dataset = sample_and_combine_datasets(datasets)

print("Training Dynamic Weighted Layer...")
trainer =  train_weighted_layer(weighted_layer, combined_dataset, tokenizer)


You shouldn't move a model that is dispatched using accelerate hooks.


Preparing Dynamic Weighted Layer...
Model architecture:
- Base model hidden size: 3584
- Number of LoRA models: 2
Sampling and Combining Datasets...
Training Dynamic Weighted Layer...


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

  0%|          | 0/66 [00:00<?, ?it/s]

C:\Users\son\AppData\Local\Temp\ipykernel_11508\4075611118.py:59: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


{'model_0_weight_mean': 0.39284974336624146, 'model_0_weight_std': 0.014491540379822254, 'epoch': 0}
{'model_1_weight_mean': 0.6071501970291138, 'model_1_weight_std': 0.01449156180024147, 'epoch': 0}
{'model_0_weight_mean': 0.369046688079834, 'model_0_weight_std': 0.03557351976633072, 'epoch': 0}
{'model_1_weight_mean': 0.6309533715248108, 'model_1_weight_std': 0.03557354211807251, 'epoch': 0}
{'model_0_weight_mean': 0.3603413701057434, 'model_0_weight_std': 0.008355570025742054, 'epoch': 0}
{'model_1_weight_mean': 0.6396586298942566, 'model_1_weight_std': 0.008355612866580486, 'epoch': 0}
{'loss': 14.0671, 'grad_norm': 0.6719631552696228, 'learning_rate': 1e-05, 'epoch': 0.3}
{'model_0_weight_mean': 0.3836636543273926, 'model_0_weight_std': 0.018038198351860046, 'epoch': 0.3}
{'model_1_weight_mean': 0.6163363456726074, 'model_1_weight_std': 0.018038218840956688, 'epoch': 0.3}
{'model_0_weight_mean': 0.3951258659362793, 'model_0_weight_std': 0.015098181553184986, 'epoch': 0.3}
{'model_

RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'lora_models.1.base_model.model.model.embed_tokens.weight', 'lora_models.0.base_model.model.model.embed_tokens.weight', 'base_model.model.embed_tokens.weight'}, {'base_model.model.layers.0.self_attn.q_proj.base_layer.weight', 'lora_models.1.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight', 'lora_models.0.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight'}, {'base_model.model.layers.0.self_attn.q_proj.base_layer.bias', 'lora_models.1.base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias', 'lora_models.0.base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias'}, {'lora_models.1.base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'base_model.model.layers.0.self_attn.q_proj.base_layer.weight.absmax', 'lora_models.0.base_model.model.model.layers.0.self_attn.q_proj.

In [15]:
trainer

NameError: name 'trainer' is not defined